In [1]:
# Librerías necesarias para realizar la práctica
# -----------------------------------------------------
# Manipulación y lectura de datos
import netCDF4 as nc
import pandas as pd
import numpy as np
from datetime import timedelta,datetime

# Estadistica
from scipy import stats

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Aquí voy a guardar cosas utiles que he aprendido en el ejercicio 1 pero que para la entrega no son necesarias

<span style='color:black'>The t-test quantifies the difference between the arithmetic means of the two samples. The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true. A p-value larger than a chosen threshold (e.g. 5% or 1%) indicates that our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. If the p-value is smaller than our threshold, then we have evidence against the null hypothesis of equal population means.</span>

In [ ]:
# Contraste de hipotesis
# Tengo que hace un t test y p test
stats.ttest_ind(std_enero_first, std_enero_last)

from statsmodels.stats.weightstats import ztest as ztest
ztest(mean_enero_first, mean_enero_last)

In [ ]:
sns.set_theme(rc={'figure.figsize':(15,6)})
sns.boxplot(data=df_1, boxprops={"facecolor": "None", "edgecolor": sns.color_palette("Paired")[1],
                          "linewidth": 0.5},
            medianprops={'color':sns.color_palette("Set2")[0]},
            whiskerprops={'color':sns.color_palette("Paired")[1]})

sns.stripplot(data=df_1, color=sns.color_palette("Paired")[1],alpha=0.3, size=1.5)

#### Boxplot CDC

In [ ]:
sns.boxplot(data=df_1, boxprops={"facecolor": "None", "edgecolor": sns.color_palette("Paired")[1],
                          "linewidth": 0.5},
            medianprops={'color':sns.color_palette("Set2")[0]},
            whiskerprops={'color':sns.color_palette("Paired")[1]})

sns.stripplot(data=df_1, color=sns.color_palette("Paired")[1],alpha=0.3, size=1.5)

## Surface temp data

In [ ]:
# Lectura de los datos 
folder = 'HadISST1'

data = nc.Dataset(f'data/{folder}/HadISST1_sst_1870-2019.nc')
lon = data.variables['longitude'][:] # data.variables.keys[0]
lat = data.variables['latitude'][:]  # data.variables.keys[1]

# Mirar los bounds: optional
# print(data.variables['lat_bnds'][:],data.variables['lon_bnds'][:])

# Datos de precipitación
precip = data.variables['sst'][:] # data.variables.keys[-1]

# Datos de tiempo: units: days since 1800-01-01 00:00:00
fecha = data.variables['time'][:]

# Convert ddata
dates = []

for data_indays in fecha:
    specific_data = datetime(1870,1,1,0,0,0)
    new_date = specific_data + timedelta(days=int(data_indays))
    dates.append(new_date)
    
# data.variables['precip'].shape = (time = 12 datos, lat = 72 datos, 
#                                   lon = 144 datos)

## Una grafica

In [6]:
import xarray as xr
import pandas as pd

folder = 'HadISST1'
ds=xr.open_dataset(f'data/{folder}/HadISST1_sst_1870-2019.nc')
month_length = ds.time.dt.days_in_month
month_length

Attributes: (0)
# Calculate the weights by grouping by 'time.season'.
weights = (
    month_length.groupby("time.season") / month_length.groupby("time.season").sum()
)


# Calculate the weighted average
ds_weighted = (ds * weights).groupby("time.season").sum(dim="time")


In [7]:
ds_unweighted = ds.groupby("time.season").mean("time")
ds_diff = ds_weighted - ds_unweighted

In [8]:
# Quick plot to show the results
notnull = pd.notnull(ds_unweighted["sst"][0])

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(20, 8))
for i, season in enumerate(("DJF","JJA")):
    ds_weighted["sst"].sel(season=season).where(notnull).plot.pcolormesh(
        ax=axes[i, 0],
        vmin=-30,
        vmax=30,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="both",
    )

    ds_unweighted["sst"].sel(season=season).where(notnull).plot.pcolormesh(
        ax=axes[i, 1],
        vmin=-30,
        vmax=30,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="both",
    )

    ds_diff["sst"].sel(season=season).where(notnull).plot.pcolormesh(
        ax=axes[i, 2],
        vmin=-0.1,
        vmax=0.1,
        cmap="RdBu_r",
        add_colorbar=True,
        extend="both",
    )

    axes[i, 0].set_ylabel(season)
    axes[i, 1].set_ylabel("")
    axes[i, 2].set_ylabel("")

for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.axis("tight")
    ax.set_xlabel("")

axes[0, 0].set_title("Weighted by DPM")
axes[0, 1].set_title("Equal Weighting")
axes[0, 2].set_title("Difference")

plt.tight_layout()

fig.suptitle('Seasonal Surface Air Temperature', fontsize=16, y=1.02)

NameError: name 'plt' is not defined

### WEB interesante

http://meteo.unican.es/work/xarray_seminar/xArray_seminar.html